<a href="https://colab.research.google.com/github/amanotk/python-resume-public/blob/master/report/GameOfLife.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" style="vertical-align: text-top;">
</a>

# 課題3. ライフゲーム (Conway's Game of Life)

2次元のセル・オートマトンとして，ライフゲーム(Conway's Game of Life)というモデルが知られている．  
詳細は[Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%82%A4%E3%83%95%E3%82%B2%E3%83%BC%E3%83%A0)などを適宜参照のこと．

2次元領域 $N_y \times N_x$ のセルの状態を $z_{i,j} (i=1, \ldots, N_y, j=1, \ldots, N_x)$ で表し，セルの状態が0の時を死，1の時を生と定義し，与えられたセルの状態から次の世代のセルの生死を次のようなルールで決定しよう．

- あるセルが死んでいる場合，そのセルに隣接する生きたセルがちょうど3つあれば，次の世代で生，そうでなければ死となる．
- あるセルが生きている場合，そのセルに隣接する生きたセルが2つか3つならば、次の世代で生，そうでなければ死となる．

このような極めて単純なルールでありながら，ライフゲームには複雑な(生命体のような)振る舞いをするパターンが多数知られている．
このライフゲームを実装してみよう．

とは言っても表示部分をイチから作るのは少し大変なので，関数 `update_cells()` を現在のセルの状態をルールに基づき更新する関数として実装するだけでよい．  
この `update_cells()` には引数として2次元配列 $z_{i,j}$ が与えられるので，更新した状態でこの配列を上書きすればよい．  
なお，$z_{i,j}$ の配列形状は $(N_y+2, N_x+2)$ となっており，両端の余分な領域は境界条件を表す「のりしろ」である．
境界条件は自動で設定されるので， `update_cells()` では $i = {1, \ldots, N_y}, j = {1, \ldots, N_x}$ の領域のみ更新すればよい．

## 注意
- 例えば"snail.lif"の動作は https://conwaylife.com/wiki/Snail で確認できる．
- jupyter-labを使っていてこのNotebookが上手く動かない人は
```bash
  $ jupyter labextension install @jupyter-widgets/jupyterlab-manager
```
によって拡張機能をインストールする必要があるかもしれない．

In [ ]:
# 定数
BASE_URL = "https://amanotk.github.io/python-resume-public/report/data/"
# LIFELIST  = 'lifelist1.html'
LIFELIST = "lifelist2.html"
LOCAL_DIR = None
N_DEFAULT = 256
BC_TYPE = 0

In [ ]:
#
# このセルはこのままにしておくこと
#
from IPython.display import display, Markdown
import ipywidgets as widgets

import numpy as np
from matplotlib import pyplot as plt

import os
import io
import re
import urllib.request
from bs4 import BeautifulSoup


mdtext = """
### 読み込み結果
{:s}

### ファイルの説明
{:s}
"""


class GameOfLife:
    # static data
    bctype = BC_TYPE
    datadir = LOCAL_DIR
    baseurl = BASE_URL
    lifeurl = baseurl + "life/"
    listurl = baseurl + LIFELIST

    def __init__(self, Ny, Nx, update):
        self.Ny = Ny
        self.Nx = Nx
        self.update = update
        self.file = None
        self.init_widgets()

    def find_file_url(self, url, ext):
        with urllib.request.urlopen(url) as response:
            html = response.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            file = []
            for link in soup.find_all("a"):
                f = link.attrs["href"]
                if re.match(r"^.+\." + ext + "$", f) is not None:
                    file.append(os.path.basename(f))
            file.sort()
            return file

    def find_file_dir(self, datadir, ext):
        file = []
        for f in os.listdir(datadir):
            if re.match(r"^.+\." + ext + "$", f) is not None:
                file.append(os.path.basename(f))
        file.sort()
        return file

    def init_widgets(self):
        maxint = np.iinfo(np.int32).max
        # UI
        play = widgets.Play(
            value=0, min=0, max=maxint, step=1, interval=100, disabled=False, show_repeat=False
        )
        label1 = widgets.Label(
            value="世代:",
            layout=widgets.Layout(width="50px", display="flex", justify_content="flex-end"),
        )
        label2 = widgets.Label(
            value="", layout=widgets.Layout(width="350px", display="flex", justify_content="center")
        )
        label3 = widgets.Output(
            layout={
                "border": "0px solid black",
                "width": "400px",
            }
        )
        select = widgets.Dropdown(
            options=["Random"],
            value="Random",
            description="初期値:",
            disabled=False,
            style=dict(description_width="50px"),
            layout=widgets.Layout(width="400px"),
        )
        self.init_select(select)
        self.ui = dict(play=play, label1=label1, label2=label2, label3=label3, select=select)
        # interactive
        output = widgets.interactive_output(self.display_plot, {"generation": play, "file": select})
        self.output = output
        # layout
        box1 = widgets.Box(
            [label1, label2],
            layout=widgets.Layout(
                display="flex", flex_flow="row", justify_content="flex-end", align_items="flex-end"
            ),
        )
        box2 = widgets.Box(
            [play, select, box1, label3],
            layout=widgets.Layout(
                display="flex",
                flex_flow="column",
                justify_content="flex-start",
                align_items="center",
            ),
        )
        box3 = widgets.Box(
            [box2, output],
            layout=widgets.Layout(
                display="flex", flex_flow="row", justify_content="center", align_items="center"
            ),
        )
        self.widget = box3

    def init_select(self, select):
        options = ["Random"]
        if self.datadir is not None:
            options.extend(self.find_file_dir(self.datadir, "lif"))
        else:
            options.extend(self.find_file_url(self.listurl, "lif"))
        select.options = options
        select.value = "Random"
        self.file = None
        select.observe(self.on_file_select, names="value")

    def on_file_select(self, change):
        if change.new == "Random":
            self.file = None
        else:
            self.file = change.new
        self.ui["play"].value = 0

    def load(self, file, ny, nx):
        data = np.zeros((ny + 2, nx + 2), np.int32)

        if file == "Random":
            # 乱数で初期化
            data[1 : ny + 1, 1 : nx + 1] = np.random.randint(0, 2, size=(ny, nx))
            with self.ui["label3"]:
                self.ui["label3"].clear_output()
        elif os.path.exists(file):
            # ローカルファイル
            with open(file, "r") as fp:
                self.read_life(fp, data)
        else:
            # Web
            with urllib.request.urlopen(self.lifeurl + file) as response:
                fp = io.StringIO(response.read().decode("utf-8"))
                self.read_life(fp, data)

        return data

    def check_read_life(self, ix, iy, nx, ny):
        if ix >= 1 and ix <= nx and iy >= 1 and iy <= ny:
            return True, ix, iy
        else:
            ix = min(max(ix, 0), nx + 1)
            iy = min(max(iy, 0), ny + 1)
            return False, ix, iy

    def read_life(self, fp, data):
        ny = data.shape[0] - 2
        nx = data.shape[1] - 2
        yoffset = ny // 2
        xoffset = nx // 2
        line = fp.readline().lstrip().rstrip()
        status = True
        description = ""
        while line:
            if len(line) >= 2 and line[0] == "#":
                if line[0:2] == "#P":
                    origin = [int(x) for x in line[2:].split()]
                    if len(origin) != 2:
                        raise ValueError("Unrecognized format")
                    yo = yoffset + origin[1]
                    xo = xoffset + origin[0]
                    s, xo, yo = self.check_read_life(xo, yo, nx, ny)
                    status &= s
                elif line[0:2] == "#D":
                    description += "{:s}\n".format(line[2:].lstrip().rstrip())
                else:
                    pass  # それ以外のコメント行は無視
            elif len(line) == 1 and line[0] == "#":
                pass
            else:
                # データ読み込み
                ss = line.replace(".", "0").replace("*", "1")
                for i in range(len(ss)):
                    ix = xo + i
                    s, ix, yo = self.check_read_life(ix, yo, nx, ny)
                    status &= s
                    data[yo, ix] = int(ss[i])
                # 境界条件
                yo = yo + 1
                s, ix, yo = self.check_read_life(ix, yo, nx, ny)
                status &= s
            # 次の行へ
            line = fp.readline().lstrip().rstrip()

        if status:
            status_string = "成功!"
        else:
            status_string = "失敗!（領域サイズが小さい?）"

        self.md = mdtext.format(status_string, description)
        with self.ui["label3"]:
            self.ui["label3"].clear_output()
            display(Markdown(self.md))

        return data

    def display_plot(self, generation, file):
        self.ui["label2"].value = "{:d}".format(generation)
        iy, ix = np.mgrid[1 : self.Ny + 1, 1 : self.Nx + 1]
        if generation == 0:
            # 初期化
            self.fig = plt.figure(figsize=(10, 10))
            self.axs = plt.subplot()
            self.axs.set_aspect("equal")
            self.axs.set_xticks([])
            self.axs.set_yticks([])
            self.data = self.load(file, self.Ny, self.Nx)
            self.set_boundary(self.data)
            self.img = plt.imshow(self.data[iy, ix], cmap=plt.cm.gray_r)
            return
        # アップデート
        self.update(self.data)
        self.set_boundary(self.data)
        self.img.set_array(self.data[iy, ix])
        display(self.fig)

    def display_app(self):
        display(self.widget)

    def set_boundary(self, data):
        ny = data.shape[0] - 2
        nx = data.shape[1] - 2
        if self.bctype == 0:
            ### 常にゼロ
            data[0, :] = 0
            data[ny + 1, :] = 0
            data[:, 0] = 0
            data[:, nx + 1] = 0
        else:
            ### 周期境界条件
            data[0, :] = data[ny, :]
            data[ny + 1, :] = data[1, :]
            data[:, 0] = data[:, nx]
            data[:, nx + 1] = data[:, 1]

In [ ]:
def update_cells(data):
    # 与えられた2次元配列dataを次の状態に更新する関数
    # dataに新しい状態を上書きすればよい
    pass


# 表示
gol = GameOfLife(N_DEFAULT, N_DEFAULT, update_cells)
gol.display_app()

## 発展編

領域が大きくなると関数 `update_cells()` の実装によってはスムーズにアニメーションが動かないだろう．  
Pythonのforループで素朴に実装した場合とNumPyの配列演算を使った場合で実行時間を比較してみよう．  
また[Namba](http://numba.pydata.org/)，[f2py](https://numpy.org/doc/stable/f2py/)，[Cython](https://cython.org/)などを使ってパフォーマンスを比較してみても面白いだろう．  
この中ではNambaがおそらく一番簡単である．Cythonはモジュールとして実装するのが必須なため少々面倒である．  

なお，手元で簡単に計測してみたところPythonのループで素朴に実装した場合に比べてNumPy配列を使った実装は約36倍高速であった．  
nambaとf2pyはほぼ同じ速度で，NumPy配列を使った場合に比べて更に約27倍高速になった．  
ただし，これは実装方法や環境に依存するものなのであくまでも目安である．

実行時間の計測には様々な方法があるが，以下のように `%timeit` を使うのが簡単だろう．

In [ ]:
ny, nx = 1024, 1024
data = np.random.randint(0, 2, size=(ny+2, nx+2))
%timeit update_cells(data)